This Python script uses the YouTube Data API to extract comments from a specific YouTube video and filters them to retain only those written in Telugu (using Unicode character range detection).

In [ ]:
import re
import pandas as pd
from googleapiclient.discovery import build

# 1. Configuration
# Set your YouTube Data API key (replace with your actual key)
API_KEY = 'AIzaSyCLJP2eVGlLn2Cuig0n34PVGzr4#######'

# Specify the YouTube video ID to extract comments from
# Example format: 'dQw4w9WgXcQ'
VIDEO_ID = 'C3o_UaQb-Es'

# Define the output Excel file name where the filtered comments will be saved
EXCEL_FILE = 'S_T_001_comments.xlsx'

# 2. Initialize YouTube client
youtube = build('youtube', 'v3', developerKey=API_KEY)

# 3. Paginate through comment threads for one video
comments = []
next_page_token = None

while True:
    response = youtube.commentThreads().list(
        part='snippet',
        videoId=VIDEO_ID,             # ← Target single video
        maxResults=100,
        pageToken=next_page_token,
        textFormat='plainText'
    ).execute()
    
    for item in response.get('items', []):
        tlc = item['snippet']['topLevelComment']
        comment_id = tlc['id']
        snippet    = tlc['snippet']
        text       = snippet['textDisplay']
        
        # 4. Filter for Telugu characters
        if re.search(r'[\u0C00-\u0C7F]', text):
            comments.append({
                'comment_id':   comment_id,
                'author':       snippet['authorDisplayName'],
                'published_at': snippet['publishedAt'],
                'text':         text.replace('\n', ' ')
            })
    
    next_page_token = response.get('nextPageToken')
    if not next_page_token:
        break

# 5. Save to Excel
df = pd.DataFrame(comments)
df.to_excel(EXCEL_FILE , index=False, sheet_name='TeluguComments') 

print(f"Saved {len(df)} Telugu comments from video {VIDEO_ID} to {EXCEL_FILE}")


Saved 146 Telugu comments from video C3o_UaQb-Es to S_T_042_comments.xlsx
